In [25]:
import numpy as np
n = 1000
np.random.seed(123)
X_0 = np.full(n, 1)
X_1 = np.random.normal(2, 5, n)
X_2 = np.random.normal(5, 2, n)
X_3 = np.random.normal(5, 5, n)

In [26]:
np.random.seed(123)
x_0 = np.full(n, 1)
x_1 = np.random.normal(4, 3, n)
x_2 = np.random.normal(3, 2, n)
x_3 = np.random.normal(2, 1, n)

In [27]:
u = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=n)

In [28]:
Betas = np.array([[0, 1, -3, 1]])
betas = np.array([[1, 4, 2, -1]])
X = np.vstack((X_0, X_1, X_2, X_3)).T
x = np.vstack((x_0, x_1, x_2, x_3)).T

In [29]:
Y = (X * Betas).sum(axis=1) + u[:, 0]
y = (x * betas).sum(axis=1) + u[:, 1]

In [30]:
import pystan

Проверим работу модели на одной итерации.

In [31]:
%%time

data = {'m': 2, 'k': X.shape[1], 'n': n, 'x': x, 'X':X, 'y': np.array([y, Y]).T} 

model = pystan.stan(file = '/Users/efremovdanil/Desktop/Bayes/2_regressions_normal.stan', data = data, chains = 1,
                    iter = 2000, seed=123)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f21e5e611952633f66942b4a682e105c NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


CPU times: user 2min 6s, sys: 1.31 s, total: 2min 8s
Wall time: 3min 8s


Значения параметров получились похожими на истинные.

In [32]:
real = np.array([1, 4, 2, -1, 0, 1, -3, 1, 1, 0.5, 1])
results = np.array([model.extract()['beta'][:, 0].mean(), model.extract()['beta'][:, 1].mean(),
                     model.extract()['beta'][:, 2].mean(), model.extract()['beta'][:, 3].mean(), 
                     model.extract()['Beta'][:, 0].mean(), model.extract()['Beta'][:, 1].mean(), 
                     model.extract()['Beta'][:, 2].mean(), model.extract()['Beta'][:, 3].mean(), 
                     model.extract()['Sigma'][:, 1, 1].mean(), model.extract()['Sigma'][:, 1, 0].mean(), 
                     model.extract()['Sigma'][:, 0, 0].mean()])

In [33]:
import pandas as pd
df = pd.DataFrame(np.array([real, results]).T, columns=['Истинные значения', 'Получившиеся значения'])
df.index=['Beta_0', 'Beta_1', 'Beta_2', 'Beta_3', 'beta_0', 'beta_1', 'beta_2', 'beta_3', 'sigma_1', 
          'cov(u_1, u_2)', 'sigma_2']
df

,Истинные значения,Получившиеся значения
Beta_0,1.0,0.824068
Beta_1,4.0,4.004442
Beta_2,2.0,2.023431
Beta_3,-1.0,-0.944687
beta_0,0.0,-0.101044
beta_1,1.0,0.995866
beta_2,-3.0,-2.997954
beta_3,1.0,1.004932
sigma_1,1.0,1.002523
"cov(u_1, u_2)",0.5,0.477918
